<a href="https://colab.research.google.com/github/kilos11/Machine_Learning-Tensorflow-/blob/main/Coloumn_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [4]:
df = pd.read_csv('/content/drive/MyDrive/100-days-of-machine-learning-main/day28-column-transformer/covid_toy.csv')

In [5]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [ ]:
X_train

,age,gender,fever,cough,city
55,81,Female,101.0,Mild,Mumbai
76,80,Male,100.0,Mild,Bangalore
22,71,Female,98.0,Strong,Kolkata
93,27,Male,100.0,Mild,Kolkata
33,26,Female,98.0,Mild,Kolkata
...,...,...,...,...,...
2,42,Male,101.0,Mild,Delhi
51,11,Female,100.0,Strong,Kolkata
5,84,Female,NaN,Mild,Bangalore
40,49,Female,102.0,Mild,Delhi


## 1. Aam Zindagi

In [9]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [8]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [10]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [11]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [12]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

## Mentos Zindagi

In [13]:
from sklearn.compose import ColumnTransformer

In [15]:
# Creating a ColumnTransformer to apply different transformers to specific columns in a DataFrame

# ('tnf1', SimpleImputer(), ['fever']):
# - Using SimpleImputer to fill missing values in the 'fever' column
# - 'tnf1' is a unique identifier for this transformer

# ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']):
# - Using OrdinalEncoder to encode categorical values in the 'cough' column with specified ordinal categories
# - 'tnf2' is a unique identifier for this transformer

# ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender', 'city']):
# - Using OneHotEncoder to perform one-hot encoding on the 'gender' and 'city' columns
# - sparse=False creates a dense array instead of a sparse matrix
# - drop='first' drops the first level of each categorical variable to avoid multicollinearity
# - 'tnf3' is a unique identifier for this transformer

# remainder='passthrough':
# - All columns not specified in the transformers will be included in the output without any transformation
# - 'passthrough' ensures that the remaining columns are included in the final transformed data without changes
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough']),
    ('tnf3', OneHotEncoder(sparse=False, drop='first'), ['gender', 'city'])
], remainder='passthrough')


In [ ]:
transformer.fit_transform(X_train).shape

(80, 7)

In [ ]:
transformer.transform(X_test).shape

(20, 7)